In [112]:
import re
import os
import ast
import pandas as pd

In [ ]:
# Folder path
paths=["exp_results"]

# Prefix to filter files
goals=['clid','ce','ce_sloss','mae']
# Initialize a list to store results
datasets=['cifar100_']
prefixs=[]
for goal in goals:
    for dataset in datasets:
        prefixs.append(f"vri_resnet18_{goal}-{dataset}")
results =[]




# Iterate over files in the folder
for folder_path in paths:
    
    for file_prefix in prefixs:
        for filename in os.listdir(folder_path):
            # Check if the file name starts with the desired prefix
            if filename.startswith(file_prefix) and 'tau' in filename:
                
                # Full file path
                file_path = os.path.join(folder_path, filename)
                print(filename)
                # Extract dataset, corruption_type, and corruption_prob from the filename
                _, rest = filename.split(file_prefix, 1)
                atts=rest.split("_")
                # dataset, corruption_type, corruption_prob= atts[:3]
                corruption_type, corruption_prob= atts[:2]
                tau=float(atts[-3].split('tau')[1])
                best_acc=-1
                best_epoch_end_acc_5 = -1
                best_epoch_end_acc_1 = -1
                best_epoch_accu_5 = -1
                best_epoch_accu_1 = -1
                best_ema_acc=-1
                best_epoch_accu_3=-1
                # Read the file and extract the best accuracy
                
                cond="epoch:[150"
                    
                with open(file_path, 'r') as file:
                    for line in file:
                        if "epoch_end:" in line:
                            match = re.search(r"epoch_end:\s*({.*})", line)
                            if match:
                                epoch_end = ast.literal_eval(match.group(1))
                                if 'acc_5' in epoch_end:
                                    best_epoch_end_acc_5 = epoch_end['acc_5']
                                if 'acc_1' in epoch_end:
                                    best_epoch_end_acc_1 = epoch_end['acc_1'] 
                        # Extract the `epoch_accu` dictionary
                        elif "epoch_accu:" in line:
                            match = re.search(r"epoch_accu:\s*({.*})", line)
                            if match:
                                epoch_accu = ast.literal_eval(match.group(1))
                                if 'acc_5' in epoch_accu:
                                    best_epoch_accu_5 = epoch_accu['acc_5']
                                if 'acc_1' in epoch_accu:
                                    best_epoch_accu_1 = epoch_accu['acc_1']
                                if 'acc_3' in epoch_accu:
                                    best_epoch_accu_3 = epoch_accu['acc_3']       
                        if "test_acc" in line:
                            test_acc_match = re.search(r"test_acc:([\d.]+)", line)
                            last_acc = float(test_acc_match.group(1))
                            match = re.search(r"epoch:\[(\d+)/(\d+)\]", line)
                            if match:
                                current_epoch = int(match.group(1))
                            if last_acc>best_acc:
                                best_acc=last_acc
                                best_epoch=current_epoch


                        if cond in line:
                            results.append({
                                "dataset": file_prefix,
                                "corruption_type": corruption_type,
                                "corruption_prob": float(corruption_prob),
                                'tau':tau,
                                "best_acc": best_acc,
                                "last_acc": last_acc,
                                "best_epoch":best_epoch,
                                "best_epoch_accu_5":best_epoch_accu_5*100,
                                "best_epoch_accu_3":best_epoch_accu_3*100,
                                "best_epoch_accu_1":best_epoch_accu_1*100,
                                "best_epoch_acc_5":best_epoch_end_acc_5*100,
                                "best_epoch_acc_1":best_epoch_end_acc_1*100,                        
                            })
                            best_acc=-1
                            best_epoch_end_acc_5 = -1
                            best_epoch_end_acc_1 = -1
                            best_epoch_accu_5 = -1
                            best_epoch_accu_1 = -1
                            best_ema_acc=-1
                            best_epoch_accu_3=-1
        

# Convert the results into a DataFrame
results_df = pd.DataFrame(results)

# Group by (dataset, corruption_type, corruption_prob) and calculate average and std
# grouped_results_ = results_df.groupby(["dataset", "corruption_type", "corruption_prob"])[["best_acc",'last_acc',"best_epoch_acc_5"]].agg(['mean', 'std']).reset_index()

# Display the grouped results


In [ ]:


results = (
    results_df.groupby(["dataset","corruption_type", "corruption_prob","tau"], group_keys=False)
)
grouped_results_ = results.groupby(["dataset","corruption_type", "corruption_prob",'tau'])[["best_acc",'last_acc',"best_epoch_acc_5"]].agg(['mean', 'std']).reset_index()

grouped_results_.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col 
                            for col in grouped_results_.columns]

# Combine mean and std into "mean ± std" format
for col in ["best_acc", "last_acc", "best_epoch_acc_5",]:
    grouped_results_[col] = (
        grouped_results_[f"{col}_mean"].round(2).astype(str) + 
        " ± " +
        grouped_results_[f"{col}_std"].round(2).astype(str)
    )

# Drop intermediate mean and std columns
grouped_results = grouped_results_.drop(
    columns=[f"{col}_mean" for col in ["best_acc", "last_acc", "best_epoch_acc_5"]] +
            [f"{col}_std" for col in ["best_acc", "last_acc", "best_epoch_acc_5"]]
)

# Transpose the DataFrame for readability
grouped_results_T = grouped_results.T


/tmp/ipykernel_2270107/3851985044.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(top3_best_acc)


In [ ]:
grouped_results_T